In [2]:
import gensim
import read_data
from pprint import pprint  # pretty-printer
from collections import defaultdict
import numpy as np

In [3]:
from preprocess_utils import get_processed_data
docs = get_processed_data(split=False)["text"].to_list()

# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
from gensim import corpora

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
docs = [doc if not pd.isnull(doc) else "" for doc in tqdm(docs) ]
for idx in tqdm(range(len(docs))):
    # print(docs[idx])
    if not pd.isnull(docs[idx]): 
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in tqdm(docs)]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in tqdm(docs)]

# Remove common words
stoplist = set('that for a of the and to in'.split())
docs = [[token for token in doc if token not in stoplist] for doc in tqdm(docs)]

# Remove words that appear only once
frequency = defaultdict(int)
for text in docs:
    for token in text:
        frequency[token] += 1

docs = [
    [token for token in text if frequency[token] > 1]
    for text in docs
]

dictionary = corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(text) for text in docs]
corpora.MmCorpus.serialize("../tmp/corpus", corpus)


/var/folders/57/vt366_td3_37y7bqh4rk7nqr0000gn/T/ipykernel_1017/762429881.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  docs = get_processed_data(split=False)["text"].to_list()
100%|██████████| 217801/217801 [00:10<00:00, 21392.09it/s]


In [4]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=30)


In [5]:
from gensim import similarities
index = similarities.Similarity("../tmp/index",lsi[corpus],num_features=30)

In [6]:
index.save('../tmp/gensim.index')
index = similarities.MatrixSimilarity.load('../tmp/gensim.index')

In [11]:
loaded_corp = corpora.MmCorpus("../tmp/corpus")
n = 10
vec = np.zeros((217801,2*n))
for i in tqdm(range(len(loaded_corp))):
    vec_lsi = lsi[loaded_corp[i]]
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    for j in range (n):
        vec[i,2*j] = sims[j+1][0]
        vec[i,2*j+1] = sims[j+1][1]

  0%|          | 483/217801 [01:07<8:22:42,  7.20it/s]


KeyboardInterrupt: 

In [8]:
data = pd.DataFrame(vec)
data.to_csv("../tmp/similGraph.csv")